In [32]:
#常规模块
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#特征工程
from sklearn.feature_selection import SelectKBest, chi2
import datetime
#模型
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

#读取数据
train=pd.read_csv(r'C:\Users\ASUS\Desktop\天池数据\train.csv')
testA=pd.read_csv(r'C:\Users\ASUS\Desktop\天池数据\testA.csv')

In [33]:
#查看特征属性唯一特征，考虑剔除--->policyCode
one_values_feature = []
for i in list(train.columns):
    if train[i].nunique() <= 1:
        one_values_feature.append(i)
for data in [train,testA]:
    data.drop(one_values_feature,axis=1,inplace=True)

#删除与也不不符合的数据（dti<0）
train.drop(list(train[train['dti']<0].index),axis=0,inplace=True)

In [34]:
#object特征数值化
#需要数值化的特征有grade，subGrade，employmentLength，issueDate，earliesCreditLine
#grade处理
for data in [train, testA]:
    data['grade'] = data['grade'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7})
#subGrade处理
for data in [train, testA]:
    data['subGrade'] = data['subGrade'].map({'A1':1,'A2':2,'A3':3,'A4':4,'A5':5,'B1':6,'B2':7,'B3':8,'B4':9,'B5':10,'C1':11,'C2':12,'C3':13,'C4':14,'C5':15,'D1':16,'D2':17,'D3':18,'D4':19,'D5':20,'E1':21,'E2':22,'E3':23,'E4':24,'E5':25,'F1':26,'F2':27,'F3':28,'F4':29,'F5':30,'G1':31,'G2':32,'G3':33,'G4':34,'G5':35})

In [35]:
#issueDate处理
for data in [train, testA]:
    data['issueDate_year'] = data.issueDate.apply(lambda x: int(x.split('-')[0]))
    data['issueDate_month'] = data.issueDate.apply(lambda x: int(x.split('-')[1]))
    data['issueDate_day'] = data.issueDate.apply(lambda x: int(x.split('-')[2]))
    data.drop(['issueDate'], axis=1, inplace=True)

In [36]:
#employmentLength处理
enc_list = {'< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4, '5 years': 5,
           '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9, '10+ years': 10}
for data in [train, testA]:
    data['employmentLength'] = data['employmentLength'].map(enc_list)

In [37]:
#earliesCreditLine处理
month_dict = {'Aug': 8, 'May': 5, 'Jul': 7, 'Oct': 10, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Nov': 11, 'Feb': 2,
                  'Mar': 3, 'Jun': 6, 'Sep': 9}
for data in [train, testA]:
    data['earliesCreditLine_year'] = data['earliesCreditLine'].apply(lambda x: int(x.split('-')[1]))
    data['earliesCreditLine_month'] = data['earliesCreditLine'].apply(lambda x: x.split('-')[0]).map(month_dict)
    data.drop(['earliesCreditLine'], axis=1, inplace=True)

In [38]:
#划分分类特征和数值特征
cate_features=['term','grade','subGrade','employmentTitle','employmentLength','homeOwnership','verificationStatus','purpose','postCode','regionCode','initialListStatus','applicationType','title','issueDate_year','issueDate_month','issueDate_day','earliesCreditLine_year','earliesCreditLine_month','delinquency_2years','ficoRangeLow','ficoRangeHigh','openAcc','pubRec','pubRecBankruptcies','revolBal','revolUtil','totalAcc']
num_features=['loanAmnt','interestRate','installment','annualIncome','dti']

In [39]:
#填充缺失值
#按照平均数填充特征
for data in [train, testA]:
    data[num_features] = data[num_features].fillna(data[num_features].mean())
    data[cate_features] = data[cate_features].fillna(train[cate_features].mean())
#匿名变量缺失值填充
n_features = ['n0', 'n1', 'n2', 'n3','n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14']
for data in [train, testA]:
    data[n_features] = data[n_features].fillna(train[n_features].mean())

In [40]:
#构建业务特征
for data in [train, testA]:
    data['avg_income'] = data['annualIncome'] / data['employmentLength']
    data['money_total'] = data['annualIncome'] * data['employmentLength']
    data['total_income'] = data['annualIncome'] * data['employmentLength']
    data['avg_loanAmnt'] = data['loanAmnt'] / data['term']
    data['all_installment'] = data['installment'] * data['term']
    data['rest_money_rate'] = data['avg_loanAmnt'] / (data['annualIncome'] + 0.1)
    data['rest_money'] = data['annualIncome'] - data['avg_loanAmnt']
    data['closeAcc'] = data['totalAcc'] - data['openAcc']
    data['ficoRange_mean'] = (data['ficoRangeHigh'] + data['ficoRangeLow']) / 2
    data['rest_pubRec'] = data['pubRec'] - data['pubRecBankruptcies']
    data['rest_Revol'] = data['loanAmnt'] - data['revolBal']
    data['dis_time'] = data['issueDate_year'] - data['earliesCreditLine_year']
    data['openAcc_totalAcc_rate'] = data['openAcc'] / data['totalAcc']
    data['end_year'] = data['issueDate_year'] + data['term']
    data['installment_term_revolBal'] = data['installment']*12*data['term']/(data['revolBal']+0.1)
    data['revolUtil_revolBal'] = data['revolUtil']/(data['revolBal']+0.1)
    data['openAcc_totalAcc'] = data['openAcc']/data['totalAcc']
    data['revolBal_loanAmnt'] = data['revolBal']/(data['loanAmnt']+0.1)
    data['revolBal_installment'] = data['revolBal']/(data['installment']+0.1)
    data['annualIncome_installment'] = data['annualIncome']/(data['installment']+0.1)

In [41]:
#删除共线特征
for data in [train, testA]:
    data.drop(['ficoRangeHigh','ficoRangeLow','n3','avg_loanAmnt','all_installment','money_total','total_income','rest_money','revolBal_installment'],axis=1,inplace=True)
#删除无用特征
for data in [train, testA]:
    data.drop(['issueDate_day'],axis=1,inplace=True)

In [42]:
#建模评估
x=train.drop(['id','isDefault'],axis=1)
y=train['isDefault']
testA_=testA.drop(['id'],axis=1)

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

model_cat_feature=['verificationStatus','purpose','regionCode','initialListStatus','applicationType','term','homeOwnership','earliesCreditLine_year']
model=CatBoostClassifier(loss_function='Logloss',
                        eval_metric='AUC',
                        learning_rate=0.05,
                        iterations=2000,
                        random_seed=2021,
                        od_type='Iter',
                        depth=6,
                        silent=True)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=2021)
clf=model.fit(x_train,y_train,eval_set=(x_test,y_test),verbose=500,cat_features=model_cat_feature)
result=clf.predict_proba(testA_)
result

0:	test: 0.6969935	best: 0.6969935 (0)	total: 743ms	remaining: 24m 46s
500:	test: 0.7350227	best: 0.7350227 (500)	total: 6m 8s	remaining: 18m 23s
1000:	test: 0.7368359	best: 0.7368359 (1000)	total: 14m	remaining: 13m 58s
1500:	test: 0.7376338	best: 0.7376399 (1487)	total: 25m 40s	remaining: 8m 31s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.7376436588
bestIteration = 1518

Shrink model to first 1519 iterations.


array([[0.92218754, 0.07781246],
       [0.70241511, 0.29758489],
       [0.42030277, 0.57969723],
       ...,
       [0.83506997, 0.16493003],
       [0.74873852, 0.25126148],
       [0.97314548, 0.02685452]])

In [31]:
result=pd.DataFrame(list(zip(testA['id'],pd.Series(result[:,1]))),columns=['id','isDefault'])
result.to_csv(r'C:\Users\ASUS\Desktop\天池数据\result1011.csv',index=False)